### Setup

In [ ]:
import pandas as pd
import ast
import os

In [ ]:
local = pd.read_csv('datasets/crawler/local_codes.csv')
foreign = pd.read_csv('datasets/crawler/foreign_codes.csv')
codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
    .reset_index(drop=True)
)

### Methods

In [ ]:
# Returns resuts table df
def getResultsTable(file):
    resultsParty = file.loc['resultsParty', 'currentResults']

    results = (
        pd.DataFrame(ast.literal_eval(resultsParty))
        .drop(['absoluteMajority', 'constituenctyCounter', 'imageKey', 'mandates', 'presidents'], axis=1)
        .rename(columns={'acronym': 'party'})
        .set_index('party')
    )

    return results

# Returns metadata df
def getResults(file, row):
    md = pd.Series(file['currentResults'])
    md.index.name = None
    md = (
        md.drop(['availableMandates', 'compensation', 'displayMessage', 'hasNoVoting', 'resultsParty'
            , 'tie', 'tieMessage', 'totalBoycotts', 'totalForeignBoycotts', 'totalLocalBoycotts'
            , 'totalMandates', 'totalParishesApproved'])
        .to_frame()
        .T
        .assign(territoryKey = row['territoryKey'])
        .set_index('territoryKey')
    )
    
    loc = (
        pd.DataFrame(
            [row['parish'], row['county'], row['district'], row['territoryKey']], index=['parish', 'county', 'district', 'territoryKey'])
        .T
        .set_index('territoryKey')
    )
    
    r = getResultsTable(file)
    r['territoryKey'], r['parish'] = row['territoryKey'], row['parish']
    r = r.reset_index().set_index(['territoryKey', 'party'])

    return r, md, loc

# Creates directories
def mkdir():
        path = '/datasets/results/' 

        working_dir = os.getcwd()
        newpath = working_dir + path

        if not os.path.exists(newpath):
            os.makedirs(newpath)

#### Teste

In [ ]:
# row = codes.iloc[0]
# results = pd.DataFrame()
# metadata = pd.DataFrame()
# location = pd.DataFrame()

# loc = row['territoryKey'].split('-')[0].capitalize()
# file = (
#     pd.read_csv('datasets/crawler/' + loc + '/' + row['district'] + '/' + row['county'] + '/' + row['parish'] + '.csv')
#     .set_index('index')
# )

# md = pd.Series(file['currentResults'])
# md.index.name = None
# md = (
#     md.drop(['availableMandates', 'compensation', 'displayMessage', 'hasNoVoting', 'resultsParty'
#         , 'tie', 'tieMessage', 'totalBoycotts', 'totalForeignBoycotts', 'totalLocalBoycotts'
#         , 'totalMandates', 'totalParishesApproved'])
#     .to_frame()
#     .T
#     .assign(territoryKey = row['territoryKey'])
#     .set_index('territoryKey')
# )

# md


### Create Results files

In [ ]:
results = pd.DataFrame()
metadata = pd.DataFrame()
location = pd.DataFrame()

for i, row in codes.iterrows():
    file = (
        pd.read_csv('datasets/crawler/' + row['territoryKey'].split('-')[0].capitalize() + '/' + row['district'] + '/' + row['county'] + '/' + row['parish'] + '.csv')
        .set_index('index')
    )
    
    r, md, loc = getResults(file, row)

    results = pd.concat([results, r])    
    metadata = pd.concat([metadata, md])
    location = pd.concat([location, loc])

In [ ]:
mkdir()

results.to_csv('datasets/results/results.csv')
metadata.to_csv('datasets/results/metadata.csv')
location.to_csv('datasets/results/location.csv')

### Explore Data

In [ ]:
results = pd.read_csv('datasets/results/results.csv')
metadata = pd.read_csv('datasets/results/metadata.csv')

parties = results['party'].unique()
results = results.set_index(['parish', 'party'])

idx = pd.IndexSlice

In [ ]:
results.loc['Angra (Sé)', 'PS']

In [ ]:
# Results by Party
# for i, party in parties:
#     results.loc[idx[:, party], idx[:]]

## Database

In [ ]:
# results
# metadata


In [ ]:
# # Send to SQL
# import mysql.connector
# from dotenv import load_dotenv

# load_dotenv()
# user = os.getenv('USER')
# password = os.getenv('PW')

# cnx = mysql.connector.connect(
#     user=user,
#     password=password,
#     host="localhost",
#     database="legislativas"
# )


